# BigDFT Fragmentation 

In this notebook we will go through the steps needed for computing the purity indicator and spillage values. First, we define the calculator we will use.

In [2]:
from BigDFT import Calculators as C

In [3]:
code = C.SystemCalculator(omp=2, mpi_run="mpirun")

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun /Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/Build/install/bin/bigdft


In [4]:
code.update_global_options(skip=True)

## Setup Calculation

First we define the system, which will be two water molecules.

In [5]:
positions ="""
- O: [-0.931477999671, 0.004547999723, -0.117331000086]
- H: [-1.355753693973, -0.032108553467, 0.725181911626]
- H: [-0.003078395366, 0.011003111969, 0.053703839371]
- O: [1.500000000000, -0.003706000000, 0.113495000000]
- H: [1.796831000000, 0.762122000000, -0.351887000000]
- H: [1.773536000000, -0.747744000000, -0.399151000000]
"""

In [6]:
from futile import YamlIO as Y
posinp = {"positions": Y.load(stream=positions, doc_lists=False),
          "units":"angstroem"}

We will next define an input file for doing a calculation. For reference, let's do a run using both the linear and the cubic scaling version.

In [7]:
from BigDFT import Inputfiles as I

In [8]:
inp = I.Inputfile()
inp.set_xc("PBE")
inp.set_hgrid(0.4)
inp.write_orbitals_on_disk()
inp.calculate_dipole()
inp["lin_basis"] = {}
inp["lin_basis"]["nit"] = 1
inp["import"] = "linear"

In [9]:
full_qm = code.run(name="linear", input=inp, posinp=posinp, 
                   run_dir="linear_dir", skip=False)

Creating the yaml input file "linear_dir/linear.yaml"
Run directory linear_dir
Executing command:  mpirun /Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/Build/install/bin/bigdft -n linear


## Fragmentation Study
First we need to divide the system up into two fragments.

In [10]:
from BigDFT import Fragments as F
reload(F)

<module 'BigDFT.Fragments' from '/Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/Build/install/lib/python2.7/site-packages/BigDFT/Fragments.pyc'>

In [11]:
dimer = F.System(mp_dict=full_qm.electrostatic_multipoles, nat_reference=3)

In [12]:
dimer.dict()
first, second = dimer.fragments

In [13]:
from BigDFT import PostProcessing as PP
reload(PP)

<module 'BigDFT.PostProcessing' from '/Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/Build/install/lib/python2.7/site-packages/BigDFT/PostProcessing.pyc'>

In [14]:
btool = PP.BigDFTool()

In [15]:
btool.get_fragment_multipoles(full_qm, dimer)

{'kernel_file': '/Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/PyBigDFT/source/notebooks/linear_dir/data-linear/density_kernel_sparse.txt', 'fragment_file': '/Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/PyBigDFT/source/notebooks/linear_dir/data-linear/fragment.dat', 'matrix_format': 'serial_text', 'multipole_matrix_1_2': '/Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/PyBigDFT/source/notebooks/linear_dir/data-linear/mpmat_1_1.txt', 'multipole_matrix_2_4': '/Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/PyBigDFT/source/notebooks/linear_dir/data-linear/mpmat_2_2.txt', 'multipole_matrix_1_0': '/Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/PyBigDFT/source/notebooks/linear_dir/data-linear/mpmat_1_-1.txt', 'multipole_matrix_1_1': '/Users/dawson/Documents/CEACollaboration/SourceCode/PurifySteps/PyBigDFT/source/notebooks/linear_dir/data-linear/mpmat_1_0.txt', 'multipole_matrix_2_1': '/Users/dawson/Document